In [41]:
import sys
from pathlib import Path

nb_dir = Path.cwd()
project_root = nb_dir.parent

sys.path.append(str(project_root))


In [42]:
from pathlib import Path

import torch
import torchaudio
import bigvgan

from utils.data import create_dataloader

ModuleNotFoundError: No module named 'bigvgan'

### Sample Dataset Config

In [ ]:
loader = create_dataloader(
    root_dir="../../data/wav",
    batch_size=1,
    segment_seconds=2.0,
    sample_rate=22050,
    n_fft=1024,
    hop_length=256,
    n_mels=80,
    return_waveform=True,
    num_workers=0,  
)

### Loading Vocos model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

batch = next(iter(loader))
mel = batch["mel"]          # [B, 1, n_mels, T]
wav = batch["waveform"]     # [B, 1, segment_samples]
paths = batch["path"]

print("Loaded file:", paths[0])
print("mel shape:", mel.shape)
print("waveform shape:", wav.shape)


model = bigvgan.BigVGAN.from_pretrained(
    "nvidia/bigvgan_v2_22khz_80band_256x",
    use_cuda_kernel=False,
).to(device)

# remove weight norm and set eval mode as recommended
model.remove_weight_norm()
model.eval()

# BigVGAN expects [B, C, T] with C = 80 mels
mel_for_vocoder = mel.squeeze(1).to(device)  # [B, 80, T]

with torch.inference_mode():
    audio_gen = model(mel_for_vocoder)       # [B, 1, num_samples]

print("Generated audio shape:", audio_gen.shape)

sr = model.h.sampling_rate  # BigVGAN’s native sampling rate (≈ 22050/22000)

print("Vocoder sample rate:", sr)

out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)

# 3) Save original chunk and vocoder output for comparison
torchaudio.save(
    out_dir / "original_chunk.wav",
    wav[0].cpu(),
    sample_rate=sr,  # or 22050, they should be very close
)

torchaudio.save(
    out_dir / "bigvgan_from_mel.wav",
    audio_gen[0].cpu(),
    sample_rate=sr,
)

print("Saved:")
print(" - outputs/original_chunk.wav")
print(" - outputs/bigvgan_from_mel.wav")

Using device: cpu


ImportError: TorchCodec is required for load_with_torchcodec. Please install torchcodec to use this function.